# Demo for Application Scorecard Model Documentation

This interactive notebook guides model developers through the process of documenting a model with the ValidMind Developer Framework. It uses the [Lending Club](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data) sample dataset from ... to build a simple application scorecard.

As part of the notebook, you will learn how to train a sample model while exploring how the documentation process works:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train an application scorecard model
- Running a ValidMind test suite to quickly generate documention about the data and model

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/register-models-in-model-inventory.html))

   For example, to register a model for use with this notebook, select:

   - Documentation template: `Binary classification`
   - Use case: `Marketing/Sales - Attrition/Churn Management`

   You can fill in other options according to your preference.

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:

In [ ]:
import pandas as pd
import numpy as np
import os
import tempfile
import shutil
import statsmodels.api as sm

%matplotlib inline

### Preview the documentation template

A template predefines sections for your model documentation and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:

In [ ]:
# Import the sample dataset from the library

from validmind.datasets.credit_risk import lending_club

raw_df = lending_club.load()
raw_df.head()

### Clean the raw dataset

Cleaning involves a series of processing steps to refine the dataset for statistical analysis. This is simplified by utilizing `lending_club.clean_data`, which implements the following data cleaning tasks: 

- Drop rows with missing target values. 
- Drop columns with more than 50% missing values. 
- Drop columns with only one unique value.

In [ ]:
clean_df = lending_club.clean(raw_df)
clean_df.head()

### Prepocess the clean dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `lending_club.preprocess` to simplify preprocessing.

In [ ]:
preprocess_df = lending_club.preprocess(clean_df)
preprocess_df.head()

### Feature engineering

TBC.

In [ ]:
fe_df = lending_club.feature_engineering(preprocess_df)
fe_df.head()

## Fit the model

TBC.

In [ ]:
train_df, test_df = lending_club.split(fe_df, add_constant=True)

x_train = train_df.drop(lending_club.target_column, axis=1)
y_train = train_df[lending_club.target_column]
x_test = test_df.drop(lending_club.target_column, axis=1)
y_test = test_df[lending_club.target_column]

# Define the model
model = sm.GLM(
    y_train, 
    x_train, 
    family=sm.families.Binomial())

# Fit the model
model = model.fit()
model.summary()

## Document the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.

### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, processed, training and test datasets (`raw_df`, `clean_df`, `preprocessed_df`, `fe_df`,  `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):

In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=lending_club.target_column,
)

vm_clean_dataset = vm.init_dataset(
    dataset=clean_df,
    input_id="clean_dataset",
    target_column=lending_club.target_column,
)

vm_preprocess_dataset = vm.init_dataset(
    dataset=preprocess_df,
    input_id="preprocess_dataset",
    target_column=lending_club.target_column,
)

vm_fe_dataset = vm.init_dataset(
    dataset=fe_df,
    input_id="fe_dataset",
    target_column=lending_club.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df,
    input_id="train_dataset",
    target_column=lending_club.target_column,
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, 
    input_id="test_dataset", 
    target_column=lending_club.target_column
)

print(vm_raw_dataset)
print(vm_clean_dataset)
print(vm_preprocess_dataset)
print(vm_fe_dataset)
print(vm_train_ds)
print(vm_test_ds)

### Initialize a model object

Additionally, you need to initialize a ValidMind model object (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):

In [ ]:
vm_model = vm.init_model(
    model,
    input_id="glm_model",
)

### Assign predictions to the datasets

We can now use the `assign_predictions()` method from the `Dataset` object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:

In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)

vm_test_ds.assign_predictions(
    model=vm_model,
)

print(vm_train_ds)
print(vm_test_ds)

### Document raw data 

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.MissingValuesBarPlot",
    inputs = {
        "dataset": vm_raw_dataset
    }
)
test.log()

### Document clean data

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_clean_dataset
    }
)
test.log()

### Document preprocessed data

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TabularDescriptionTables",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.IQROutliersTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.ClassImbalance",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TabularNumericalHistograms",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TabularCategoricalBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.TargetRateBarPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "default_column": lending_club.target_column,
        "columns": None
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.PearsonCorrelationMatrix",
    inputs = {
        "dataset": vm_preprocess_dataset
    }
)
test.log()

### Document WoE analysis

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.WOEBinTable",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.data_validation.WOEBinPlots",
    inputs = {
        "dataset": vm_preprocess_dataset
    },
    params = {
        "breaks_adj": lending_club.breaks_adj
    }
)
test.log()

### Document model fit

In [ ]:
test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionCoeffsPlot",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.RegressionModelsCoeffs",
    inputs = {
        "models": [vm_model]
    }
)
test.log()

### Document model evaluation

In [ ]:
test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.GINITable",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:
run = False
if run:

    test= vm.tests.run_test(
        "validmind.model_validation.statsmodels.LogRegressionConfusionMatrix",
        inputs = {
            "model": vm_model
        },
        params = {
            "cut_off_threshold": 0.5
        }
    )
    test.log()

In [ ]:
run = False
if run:

    test= vm.tests.run_test(
        "validmind.model_validation.sklearn.ROCCurve",
        inputs = {
            "model": vm_model,
            "dataset": vm_test_ds
        }
    )
    test.log()

In [ ]:
test= vm.tests.run_test(
    "validmind.model_validation.statsmodels.LogisticRegPredictionHistogram",
    inputs = {
        "model": vm_model,
        "datasets": [vm_train_ds, vm_test_ds]
    }
)
test.log()

In [ ]:
run = True
if run:

    test= vm.tests.run_test(
        "validmind.model_validation.statsmodels.LogisticRegCumulativeProb",
        inputs = {
            "model": vm_model,
            "datasets": [vm_train_ds, vm_test_ds]
        }
    )
    test.log()

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the test results as part of your model documentation right in the ValidMind Platform UI: 

1. In the [Platform UI](https://app.prod.validmind.ai), go to the **Documentation** page for the model you registered earlier.

2. TBC

What you can see now is a more easily consumable version of the tests you just executed, along with other parts of your model documentation that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).